In [1]:
import pandas as pd
import datetime as dt
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
from plotly.subplots import make_subplots

from functions import string_datediff, sunburst_data

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
beds_ab = pd.read_csv('data-ingest/data/bed_use_betsi_cadwaladr.csv', parse_dates = [0])
beds_ab.head()

,date,indicator,count
0,2020-04-01,CO_GABed_C19_Con,21
1,2020-04-01,CO_GABed_C19_Sus,72
2,2020-04-01,CO_GABed_NonC19,656
3,2020-04-01,CO_GABed_Spare,499
4,2020-04-01,CO_IVBed_C19_Con,8


In [3]:
lockdowns = pd.read_csv('data-ingest/data/lockdown_status_bc.csv', parse_dates = [1]).drop(columns = ['health_board'])
lockdowns

,date,hb_pop,num_lockdown,prop_lockdown
0,2020-10-01,699559,504956,0.72182
1,2020-10-23,699559,699559,2.00000


In [8]:
colors = ['#747875', '#50869D', '#50869D', '#ED8103', '#ED8103']
statuses = ['(?)',  'IV_NonC19', 'GA_NonC19', 'IV_C19', 'GA_C19']

startdate = "2020-09-08"
titles = ("First day", "22nd day", "44th day")
name = 0

fig = make_subplots(rows=1, cols=3, subplot_titles = titles)

for day,dom in zip([string_datediff(startdate, x) for x in [0, 22, 44]],
                          [[0, 0.3], [0.35, 0.65], [0.7, 1]]):
    data = px.sunburst(sunburst_data(beds_ab, day), 
                       path = ['bed_type', 'bed_status', 'indicator'], 
                       values = 'count', color = 'bed_status',
                       color_discrete_map = {stat: col for stat, col in zip(statuses, colors)}
    )._data
    trace = go.Sunburst(
        name = titles[name],
        labels=data[0]['labels'],
        parents=[x.split('/')[1] if len(x.split('/')) > 1 else x for x in data[0]['parents'].tolist()],
        values=data[0]['values'],
        domain={'x': dom, 'y': [0.2, 0.95]},
        marker = dict(colors = data[0]['marker']['colors']),
        branchvalues = 'total',
        insidetextorientation='horizontal'
    )
    fig.add_trace(trace)
    name += 1

fig.update_layout(title_text="Bed use in Aneurin Bevan Health Board during local lockdown period",
                 #uniformtext=dict(minsize=10, mode='hide')
                 )

labcolors = ['#50869D', '#ED8103', '#747875']
for i, lab in enumerate(['Spare beds / Other patients', 'COVID-19 patients', 'IV = "Invasive / Ventilated";\t\tGA = "General / Acute"']):
    fig.add_annotation(dict(font=dict(color=labcolors[i],size=12),
                                        x=0,
                                        y= 0+i/20,
                                        showarrow=False,
                                        text=lab,
                                        textangle=0,
                                        xanchor='left',
                                        xref="paper",
                                        yref="paper"))
    
fig.write_html('visualisations/AB_beds_lockdown.html')
fig.show()

In [5]:
colors = ['#747875', '#50869D', '#50869D', '#ED8103', '#ED8103']
statuses = ['(?)',  'IV_NonC19', 'GA_NonC19', 'IV_C19', 'GA_C19']

startdate = "2020-10-23"

fig = make_subplots(rows=1, cols=2, subplot_titles = ("First day", "30th day"))

for day,dom in zip([string_datediff(startdate, x) for x in [0, 21]], [[0, 0.5], [0.5, 1]]):
    data = px.sunburst(sunburst_data(beds_ab, day), 
                       path = ['bed_type', 'bed_status', 'indicator'], 
                       values = 'count', color = 'bed_status',
                       color_discrete_map = {stat: col for stat, col in zip(statuses, colors)}
    )._data
    trace = go.Sunburst(
        labels=data[0]['labels'],
        parents=[x.split('/')[1] if len(x.split('/')) > 1 else x for x in data[0]['parents'].tolist()],
        values=data[0]['values'],
        domain={'x': dom, 'y': [0.2, 0.95]},
        marker = dict(colors = data[0]['marker']['colors']),
        branchvalues = 'total',
        insidetextorientation='horizontal'
    )
    fig.add_trace(trace) 

fig.update_layout(height=600, width=1000,
                  title_text="Bed use in Betsi Cadwaladr Health Board during and after firebreak lockdown period",
                 #uniformtext=dict(minsize=10, mode='hide')
                 )

labcolors = ['#50869D', '#ED8103', '#747875']
for i, lab in enumerate(['Spare beds / Other patients', 'COVID-19 patients', 'IV = "Invasive / Ventilated";\t\tGA = "General / Acute"']):
    fig.add_annotation(dict(font=dict(color=labcolors[i],size=12),
                                        x=0,
                                        y= 0+i/20,
                                        showarrow=False,
                                        text=lab,
                                        textangle=0,
                                        xanchor='left',
                                        xref="paper",
                                        yref="paper"))
    
fig.write_html('visualisations/BC_beds_firebreak.html')
fig.show()

In [6]:
beds_ab.tail()

,date,indicator,count
2215,2020-11-19,CO_IVBed_C19_Con,3
2216,2020-11-19,CO_IVBed_C19_Rec,0
2217,2020-11-19,CO_IVBed_C19_Sus,0
2218,2020-11-19,CO_IVBed_NonC19,25
2219,2020-11-19,CO_IVBed_Spare,15
